In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
chrome_data_wasm = {'normal': [88, 85, 84, 85, 89, 86, 83, 85, 84, 83, 89, 83, 85, 86, 88, 85, 81, 84, 84, 82, 87, 83, 84, 86, 88, 85, 82, 85, 84, 82, 86, 83, 84, 86, 88, 85, 82, 83, 85, 82, 88, 92, 84, 87, 90, 85, 82, 84, 84, 83], 'eager': [88, 85, 85, 87, 88, 85, 82, 85, 85, 81, 88, 85, 86, 87, 89, 86, 83, 83, 83, 93, 87, 85, 85, 87, 92, 84, 80, 83, 83, 82, 88, 85, 85, 86, 89, 83, 81, 83, 85, 82, 88, 84, 84, 87, 87, 85, 82, 84, 84, 83]}
chrome_data_nowasm = {'normal': [1444, 1416, 1424, 1406, 1406, 1409, 1415, 1408, 1576, 1554, 1434, 1414, 1419, 1407, 1405, 1415, 1417, 1405, 1549, 1555, 1437, 1414, 1423, 1413, 1408, 1411, 1417, 1406, 1405, 1407, 1439, 1417, 1428, 1410, 1406, 1408, 1411, 1408, 1402, 1402, 1442, 1440, 1447, 1407, 1404, 1409, 1410, 1400, 1397, 1395], 'eager': [1444, 1420, 1424, 1417, 1403, 1404, 1407, 1406, 1396, 1399, 1435, 1458, 1427, 1412, 1407, 1412, 1411, 1421, 1420, 1401, 1429, 1411, 1418, 1408, 1413, 1402, 1401, 1402, 1396, 1398, 1438, 1414, 1421, 1416, 1416, 1421, 1409, 1408, 1404, 1403, 1429, 1410, 1422, 1405, 1407, 1425, 1404, 1403, 1403, 1398]}
firefox_data_wasm = {'normal': [101, 103, 105, 104, 103, 100, 104, 105, 101, 103, 101, 103, 104, 104, 100, 106, 105, 106, 101, 102, 120, 101, 104, 102, 102, 100, 102, 114, 97, 100, 98, 101, 102, 104, 104, 100, 99, 103, 99, 101, 101, 102, 104, 105, 104, 101, 106, 107, 102, 100], 'eager': [104, 104, 104, 104, 109, 101, 103, 106, 100, 102, 103, 102, 106, 107, 100, 99, 105, 108, 101, 105, 101, 105, 106, 105, 106, 100, 102, 106, 103, 101, 107, 103, 105, 104, 105, 102, 107, 105, 102, 102, 97, 102, 102, 104, 115, 100, 100, 103, 100, 97]}
firefox_data_nowasm = {'normal': [1715, 1713, 1720, 1717, 1714, 1712, 1712, 1713, 1720, 1721, 1712, 1718, 1715, 1719, 1714, 1714, 1720, 1725, 1720, 1710, 1696, 1716, 1712, 1720, 1717, 1714, 1719, 1708, 1711, 1708, 1717, 1717, 1715, 1737, 1752, 1724, 1711, 1717, 1708, 1720, 1708, 1716, 1718, 1712, 1720, 1716, 1716, 1710, 1717, 1707], 'eager': [1711, 1716, 1721, 1741, 1720, 1718, 1722, 1706, 1709, 1714, 1700, 1716, 1715, 1719, 1708, 1711, 1707, 1709, 1726, 1711, 1732, 1722, 1717, 1732, 1711, 1714, 1710, 1708, 1708, 1722, 1712, 1731, 1784, 1769, 1712, 1712, 1734, 1717, 1731, 1717, 1715, 1716, 1714, 1719, 1714, 1713, 1712, 1709, 1713, 1715]}

In [ ]:
plt.scatter(range(50), chrome_data_wasm['normal'], label="chrome_wasm")
plt.scatter(range(50), firefox_data_wasm['normal'], label="firefox_wasm")
plt.legend()
plt.show()

In [ ]:
plt.scatter(range(50), chrome_data_nowasm['normal'], label="chrome_nowasm")
plt.scatter(range(50), firefox_data_nowasm['normal'], label="firefox_nowasm")
plt.legend()
plt.show()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import json


def print_timeline_data(url: str):
    caps = DesiredCapabilities.CHROME.copy()
    caps["goog:loggingPrefs"] = {"performance": "ALL"}

    driver = webdriver.Chrome(desired_capabilities=caps)
    driver.get(url)

    events = [json.loads(log["message"])["message"] for log in driver.get_log("performance")]
    navigation_start = 0
    load_event_end = 0

    for event in events:
        if event["method"] == "Network.requestWillBeSent":
            navigation_start = int(event["params"]["timestamp"] * 1000)
        elif event["method"] == "Page.loadEventFired":
            load_event_end = int(event["params"]["timestamp"] * 1000)

    print(f"Page load time for {url}: {load_event_end - navigation_start} ms")

    driver.quit()


if __name__ == "__main__":
    url = "https://www.google.com"
    print_timeline_data(url)


In [4]:
import json
import time

import ChromeController as cri

def generate_profile(url: str):
    with cri.connect() as chrome:
        # Create a new tab and enable the necessary domains
        tab = chrome.Target.createTarget(url)
        tab_id = tab["targetId"]
        chrome.Runtime.enable()
        chrome.Performance.enable()
        chrome.Network.enable()

        # Navigate to the URL and wait for the page to load
        chrome.Page.navigate(url=url, targetId=tab_id)
        chrome.wait_event("Page.loadEventFired", timeout=10)

        # Get the performance profile data
        profile = chrome.Performance.getMetrics()
        profile_data = json.dumps(profile, indent=2)

        # Save the profile to a file
        timestamp = int(time.time())
        filename = f"profile_{url}_{timestamp}.json"
        with open(filename, "w") as f:
            f.write(profile_data)

        # Close the tab and return the filename
        chrome.Target.closeTarget(targetId=tab_id)
        return filename


if __name__ == "__main__":
    url = "https://www.google.com"
    profile_filename = generate_profile(url)
    print(f"Performance profile saved to {profile_filename}")


AttributeError: __enter__

In [3]:
import json
import time

from chrome_remote import ChromeRemote


def print_timeline_data(url: str):
    with ChromeRemote() as chrome:
        tab = chrome.tabs.new()
        tab.Network.enable()
        tab.Page.enable()
        tab.Tracing.start(categories=["-*", "devtools.timeline"], options={"recordMode": "recordUntilFull"})
        tab.Page.navigate(url=url)
        tab.wait(5)

        tracing_data = tab.Tracing.stop()
        events = json.loads(tracing_data)["traceEvents"]

        for event in events:
            if event["name"] == "navigationStart":
                start_time = event["ts"]
            elif event["name"] == "loadEventEnd":
                end_time = event["ts"]
                break

        print(f"Page load time for {url}: {end_time - start_time} ms")


if __name__ == "__main__":
    url = "https://www.google.com"
    print_timeline_data(url)


ModuleNotFoundError: No module named 'chrome_remote'